## Imports

Import things needed for Tensorflow and CoreML

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __builtin__ import any as b_any

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import numpy as np
from PIL import Image

import tensorflow as tf

import configuration
import inference_wrapper
import sys
sys.path.insert(0, 'im2txt/inference_utils')
sys.path.insert(0, 'im2txt/ops')
import caption_generator
import image_processing
import vocabulary

import urllib, os, sys, zipfile
from os.path import dirname
from tensorflow.core.framework import graph_pb2
from tensorflow.python.tools.freeze_graph import freeze_graph
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile
import tfcoreml
import configuration
from coremltools.proto import NeuralNetwork_pb2

In [2]:
# Turn on debugging on error
%pdb on

Automatic pdb calling has been turned ON


## Create the models

Create the Tensorflow model and strip all unused nodes

In [3]:
checkpoint_file = './trainlogIncNEW/model.ckpt'
pre_frozen_model_file = './frozen_model_textgenNEW.pb'
frozen_model_file = './frozen_model_textgenNEW.pb'

# Which nodes we want to input for the network
# Use ['image_feed'] for just Memeception
input_node_names = ['seq_embeddings','lstm/state_feed']

# Which nodes we want to output from the network
# Use ['lstm/initial_state'] for just Memeception
output_node_names = ['softmax','lstm/state']

# Set the depth of the beam search
beam_size = 2

In [4]:
# Build the inference graph.

g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(),
                                               checkpoint_file)
g.finalize()

INFO:tensorflow:Building model.
About to decide if splitting
splitting mat mul
(1, 812)
Tensor("lstm/basic_lstm_cell/concat:0", shape=(1, 812), dtype=float32)
(812, 2048)
<tf.Variable 'lstm/basic_lstm_cell/kernel:0' shape=(812, 2048) dtype=float32_ref>
___
FIRST (1, 2048)
{'num_or_size_splits': 4, 'value': <tf.Tensor 'lstm/basic_lstm_cell/BiasAdd:0' shape=(1, 2048) dtype=float32>, 'axis': 1}
new_h Tensor("lstm/basic_lstm_cell/Mul_2:0", shape=(1, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_1:0' shape=(1, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_2:0' shape=(1, 512) dtype=float32>)
About to decide if splitting
splitting mat mul
(2, 812)
Tensor("lstm/basic_lstm_cell/Squeeze_8:0", shape=(2, 812), dtype=float32)
(812, 2048)
<tf.Variable 'lstm/basic_lstm_cell/kernel:0' shape=(812, 2048) dtype=float32_ref>
___
('NUMSPLITS', 1)
{'num_or_size_splits': 1, 'value': <tf.Tensor 'lstm/basic_lstm_cell/Squeeze_8:0' shape=(2, 812) dtype=float32

In [5]:
# Write the graph

tf_model_path = './log/pre_graph_textgenNEW.pb'
tf.train.write_graph(
    g,
    './log',
    'pre_graph_textgenNEW.pb',
    as_text=False,
)

with open(tf_model_path, 'rb') as f:
    serialized = f.read()
tf.reset_default_graph()
original_gdef = tf.GraphDef()
original_gdef.ParseFromString(serialized)

47085143

In [6]:
# Strip unused graph elements and serialize the output to file

gdef = strip_unused_lib.strip_unused(
        input_graph_def = original_gdef,
        input_node_names = input_node_names,
        output_node_names = output_node_names,
        placeholder_type_enum = dtypes.float32.as_datatype_enum)
# Save it to an output file
with gfile.GFile(pre_frozen_model_file, 'wb') as f:
    f.write(gdef.SerializeToString())

In [7]:
# Freeze the graph with checkpoint data inside

freeze_graph(input_graph=pre_frozen_model_file,
             input_saver='',
             input_binary=True,
             input_checkpoint=checkpoint_file,
             output_node_names=','.join(output_node_names),
             restore_op_name='save/restore_all',
             filename_tensor_name='save/Const:0',
             output_graph=frozen_model_file,
             clear_devices=True,
             initializer_nodes='')

INFO:tensorflow:Restoring parameters from ./trainlogIncNEW/model.ckpt
INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.


## Verify the model

Check that it is producing legit captions for *One does not simply*

In [8]:
# Configure the model and load the vocab

config = configuration.ModelConfig()

vocab_file ='vocab4.txt'
vocab = vocabulary.Vocabulary(vocab_file)

INFO:tensorflow:Initializing vocabulary from file: vocab4.txt
INFO:tensorflow:Created vocabulary with 38521 words


In [9]:
# Generate captions on a hard-coded image

with tf.Session(graph=g) as sess:
  restore_fn(sess)
  generator = caption_generator.CaptionGenerator(
      model, vocab, beam_size=beam_size)
  for i,filename in enumerate(['memes/one-does-not-simply.jpg']):
    with tf.gfile.GFile(filename, "rb") as f:
      image = Image.open(f)
      image = ((np.array(image.resize((299,299)))/255.0)-0.5)*2.0
    for k in range(5):
      captions = generator.beam_search(sess, image)    
      for i, caption in enumerate(captions):
        sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
        sentence = " ".join(sentence)
        print(sentence)

INFO:tensorflow:Loading model from checkpoint: ./trainlogIncNEW/model.ckpt
INFO:tensorflow:Restoring parameters from ./trainlogIncNEW/model.ckpt
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt
one does not simply get that a meme
one does not simply just read the first class
one does not simply get a text
one does not simply get a guy named the name of skrillex
one does not simply use the wrong meme
one does not simply pause a meme on facebook
one does not simply just eat a chicken without crust
one does not simply just find the same thing
one does not simply kill them
one does not simply find a joke in the internet


## Convert the model to CoreML

Specify output variables from the graph to be used

In [10]:
# Define basic shapes
# If using Memeception, add 'image_feed:0': [299, 299, 3]
input_tensor_shapes = {
    'seq_embeddings:0': [1, beam_size, 300],
    'lstm/state_feed:0': [1, beam_size, 1024],
}

coreml_model_file = './Textgen_NEW.mlmodel'

In [11]:
output_tensor_names = [node + ':0' for node in output_node_names]

In [ ]:
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file, 
        mlmodel_path=coreml_model_file, 
        input_name_shape_dict=input_tensor_shapes,
        output_feature_names=output_tensor_names,
        add_custom_layers=True,
)

Shapes not found for 54 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob lstm/state_feed:0
1/85: Analysing op name: seq_embeddings ( type:  Placeholder )
Skipping name of placeholder
2/85: Analysing op name: lstm/basic_lstm_cell/kernel ( type:  Const )
3/85: Analysing op name: lstm/basic_lstm_cell/kernel/read ( type:  Identity )
4/85: Analysing op name: lstm/basic_lstm_cell/bias ( type:  Const )
5/85: Analysing op name: lstm/basic_lstm_cell/bias/read ( type:  Identity )
6/85: Analysing op name: lstm/state_feed ( type:  Placeholder )
Skipping name of placeholder
7/85: Analysing op name: lstm/ExpandDims/dim ( type:  Const )
8/85: Analysing op name: lstm/ExpandDims ( type:  ExpandDims )
9/85: Analysing op name: lstm/split/split_dim ( type:  Const )
10/85: Analysing op name: lstm/split ( type:  Split )
11/85: Analysing op name: lstm/Squeeze ( type:  Squeeze )
12/85: Analysing op name: lstm/Squeeze_1 ( type:  Squeeze )
13/85: Analysing o

RuntimeError: Error compiling model: "compiler error:  Inner product layer: 'lstm/basic_lstm_cell/MatMul_1' : Product of input blob dimensions C,H,W (812,1,2) must be equal to the parameter 'inputChannels' (812)
".

> /Users/freedmand/scraps/danklearning2/venv/lib/python2.7/site-packages/coremltools/models/model.py(77)_get_proxy_from_spec()
     75             return None
     76 
---> 77         return _MLModelProxy(filename)
     78     else:
     79         return None



In [ ]:
p input_name